## Initial Setup

In [1]:
import pandas as pd 
import numpy as np 
#import matplotlib.pyplot as plt 
#import seaborn as sns 
from datetime import datetime

### Load datasets into pandas dataframe

In [2]:
customers_df = pd.read_csv("/Users/dev/python1_cogitate/data/customers_sample.csv")
policies_df = pd.read_csv("/Users/dev/python1_cogitate/data/policies_sample.csv")
claims_df = pd.read_csv("/Users/dev/python1_cogitate/data/claims_sample.csv")
fraud_df = pd.read_csv("/Users/dev/python1_cogitate/data/fraud_detection_sample.csv")

customers_df.shape, policies_df.shape, claims_df.shape, fraud_df.shape

((50, 6), (75, 8), (120, 7), (40, 6))

In [3]:
customers_df.head()

,customer_id,name,age,email,city,registration_date
0,1,Kenneth Davis,56,customer1@email.com,New York,2023-12-30
1,2,Deborah Moore,46,customer2@email.com,Los Angeles,2023-07-18
2,3,Joseph Rodriguez,60,customer3@email.com,Philadelphia,2022-12-31
3,4,Michael Baker,38,customer4@email.com,Philadelphia,2020-05-01
4,5,Paul Garcia,36,customer5@email.com,Phoenix,2023-05-23


In [4]:
policies_df.head()

,policy_id,customer_id,policy_number,policy_type,annual_premium,coverage_amount,start_date,status
0,201,29,P1001,HOME,1153.05,10385.62,2022-06-05,EXPIRED
1,202,44,P1002,HEALTH,2840.23,31689.58,2022-08-16,EXPIRED
2,203,35,P1003,BUSINESS,7113.81,64808.93,2021-08-19,ACTIVE
3,204,28,P1004,AUTO,1330.08,41918.31,2023-07-19,ACTIVE
4,205,23,P1005,HEALTH,4083.91,109910.68,2022-11-15,ACTIVE


In [5]:
claims_df.head()

,claim_id,policy_id,policy_number,claim_date,claim_amount,claim_type,status
0,101,244,P1044,2025-02-22,9644.43,MEDICAL,PENDING
1,102,203,P1003,2022-10-18,7061.45,FIRE,PENDING
2,103,271,P1071,2022-11-21,17933.97,ACCIDENT,PENDING
3,104,230,P1030,2025-05-19,8072.04,OTHER,APPROVED
4,105,229,P1029,2022-02-20,40902.59,MEDICAL,APPROVED


In [6]:
fraud_df.head()

,detection_id,claim_id,is_fraudulent,detected_by,detection_date,confidence_score
0,1,159,False,NaN,2024-06-27,NaN
1,2,141,False,NaN,2023-05-30,NaN
2,3,197,False,NaN,2022-05-09,NaN
3,4,110,False,NaN,2022-09-06,NaN
4,5,102,False,NaN,2022-10-27,NaN


## Cleaning Customers Data

In [7]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_id        50 non-null     int64 
 1   name               50 non-null     object
 2   age                50 non-null     int64 
 3   email              50 non-null     object
 4   city               50 non-null     object
 5   registration_date  50 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.5+ KB


In [8]:
customers_df.duplicated().sum()

np.int64(0)

### Observations:

1. No null values
2. No duplicates
3. Registeration date to be converted to datetime format to enable date operations, easier analysis

In [9]:
customers_df["registration_date"] = pd.to_datetime(customers_df["registration_date"], errors='coerce')

In [10]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        50 non-null     int64         
 1   name               50 non-null     object        
 2   age                50 non-null     int64         
 3   email              50 non-null     object        
 4   city               50 non-null     object        
 5   registration_date  50 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 2.5+ KB


### create a new feature for customer tenure in days

In [11]:
current_date = pd.to_datetime('today')
customers_df['customer_tenure_days'] = (current_date - customers_df['registration_date']).dt.days
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           50 non-null     int64         
 1   name                  50 non-null     object        
 2   age                   50 non-null     int64         
 3   email                 50 non-null     object        
 4   city                  50 non-null     object        
 5   registration_date     50 non-null     datetime64[ns]
 6   customer_tenure_days  50 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 2.9+ KB


Everything seems good after manual check too.

## Cleaning claims data

In [12]:
claims_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   claim_id       120 non-null    int64  
 1   policy_id      120 non-null    int64  
 2   policy_number  120 non-null    object 
 3   claim_date     120 non-null    object 
 4   claim_amount   120 non-null    float64
 5   claim_type     120 non-null    object 
 6   status         120 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 6.7+ KB


In [13]:
claims_df.duplicated().sum()

np.int64(0)

In [14]:
claims_df['claim_date'] = pd.to_datetime(claims_df['claim_date'], errors='coerce')
claims_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   claim_id       120 non-null    int64         
 1   policy_id      120 non-null    int64         
 2   policy_number  120 non-null    object        
 3   claim_date     120 non-null    datetime64[ns]
 4   claim_amount   120 non-null    float64       
 5   claim_type     120 non-null    object        
 6   status         120 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 6.7+ KB


In [15]:
claims_df['policy_id'].value_counts().sort_values(ascending=False)

policy_id
208    7
231    4
275    4
232    4
213    4
      ..
251    1
238    1
227    1
254    1
239    1
Name: count, Length: 62, dtype: int64

## cleaning policies_sample data

In [16]:
policies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   policy_id        75 non-null     int64  
 1   customer_id      75 non-null     int64  
 2   policy_number    75 non-null     object 
 3   policy_type      75 non-null     object 
 4   annual_premium   75 non-null     float64
 5   coverage_amount  75 non-null     float64
 6   start_date       75 non-null     object 
 7   status           75 non-null     object 
dtypes: float64(2), int64(2), object(4)
memory usage: 4.8+ KB


In [17]:
policies_df['start_date'] = pd.to_datetime(policies_df['start_date'], errors='coerce')
policies_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   policy_id        75 non-null     int64         
 1   customer_id      75 non-null     int64         
 2   policy_number    75 non-null     object        
 3   policy_type      75 non-null     object        
 4   annual_premium   75 non-null     float64       
 5   coverage_amount  75 non-null     float64       
 6   start_date       75 non-null     datetime64[ns]
 7   status           75 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 4.8+ KB


In [18]:
policies_df.duplicated().sum()

np.int64(0)

In [19]:
policies_df['customer_id'].value_counts().sort_values(ascending=False)
a = set(customers_df["customer_id"].unique())
b = set(policies_df['customer_id'].unique())
print("customers with no policy",int(len(a-b)))

customers with no policy 9


#### Note that multiple customers can have multiple policies

## cleaning fraud_detection data

In [20]:
fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   detection_id      40 non-null     int64  
 1   claim_id          40 non-null     int64  
 2   is_fraudulent     40 non-null     bool   
 3   detected_by       3 non-null      object 
 4   detection_date    40 non-null     object 
 5   confidence_score  3 non-null      float64
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 1.7+ KB


In [21]:
fraud_df['detection_date'] = pd.to_datetime(fraud_df['detection_date'], errors='coerce')

In [22]:
fraud_df.sample(20)

,detection_id,claim_id,is_fraudulent,detected_by,detection_date,confidence_score
15,16,145,False,NaN,2025-01-02,NaN
17,18,132,False,NaN,2023-02-20,NaN
8,9,113,False,NaN,2023-03-25,NaN
2,3,197,False,NaN,2022-05-09,NaN
18,19,148,False,NaN,2022-06-17,NaN
5,6,220,False,NaN,2022-11-30,NaN
28,29,216,False,NaN,2023-04-01,NaN
4,5,102,False,NaN,2022-10-27,NaN
3,4,110,False,NaN,2022-09-06,NaN
25,26,179,False,NaN,2023-01-14,NaN


### Observation:

1. For all customers for which claim is not fraudulent, the values for detected_by and confidence score is a NaN value
2. We must impute with meaningful placeholders to hold context.


For detected_by (Categorical): The absence of a detector means no fraud was found. We can fill the NaN values with a clear, descriptive string like 'No Fraud Detected'. This transforms the missing data into a meaningful category.

For confidence_score (Numerical): A confidence score is only relevant when fraud is suspected. For non-fraudulent claims, the "confidence of fraud" is logically zero. Therefore, we should fill the NaN values with 0.



In [23]:
fill_values = {
    'detected_by': 'No Fraud Detected', 
    'confidence_score': 0
}
fraud_df.fillna(value=fill_values, inplace=True)
print(fraud_df.head(5))

   detection_id  claim_id  is_fraudulent        detected_by detection_date  \
0             1       159          False  No Fraud Detected     2024-06-27   
1             2       141          False  No Fraud Detected     2023-05-30   
2             3       197          False  No Fraud Detected     2022-05-09   
3             4       110          False  No Fraud Detected     2022-09-06   
4             5       102          False  No Fraud Detected     2022-10-27   

   confidence_score  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  


## Merge the datasets for further analysis

### First we have to aggregate them before merging to create neat summary numbers

In [25]:
# Policy metrics by customer
policy_metrics = policies_df.groupby('customer_id').agg(
    total_policies=('policy_id', 'count'),
    active_policies=('status', lambda x: (x == 'ACTIVE').sum()),
    avg_annual_premium=('annual_premium', 'mean'),
    total_coverage=('coverage_amount', 'sum'),
    policy_types=('policy_type', lambda x: x.value_counts().to_dict()),
    policy_ids=('policy_id', list)
).reset_index()

policy_metrics.head()

,customer_id,total_policies,active_policies,avg_annual_premium,total_coverage,policy_types,policy_ids
0,1,1,1,4598.760,118547.63,{'HEALTH': 1},[256]
1,3,1,1,5543.190,121194.54,{'BUSINESS': 1},[257]
2,5,2,0,4145.435,130005.45,"{'LIFE': 1, 'BUSINESS': 1}","[259, 261]"
3,6,2,2,3863.275,226691.15,"{'BUSINESS': 1, 'HEALTH': 1}","[211, 221]"
4,7,1,1,1008.730,33545.45,{'HOME': 1},[245]


In [26]:
# Claims metrics by policy
claims_metrics = claims_df.groupby('policy_id').agg(
    total_claims=('claim_id', 'count'),
    total_claimed_amount=('claim_amount', 'sum'),
    avg_claim_amount=('claim_amount', 'mean'),
    claim_statuses=('status', lambda x: x.value_counts().to_dict())
).reset_index()

claims_metrics.head()

,policy_id,total_claims,total_claimed_amount,avg_claim_amount,claim_statuses
0,201,2,4730.78,2365.39,{'APPROVED': 2}
1,202,1,3369.93,3369.93,{'PENDING': 1}
2,203,1,7061.45,7061.45,{'PENDING': 1}
3,205,1,8567.58,8567.58,{'APPROVED': 1}
4,206,1,16401.83,16401.83,{'APPROVED': 1}


In [27]:
# Fraud metrics by claim
fraud_metrics = fraud_df.groupby('claim_id').agg(
    fraud_detected=('is_fraudulent', 'max'),
    detection_count=('is_fraudulent', 'sum'),
    avg_confidence=('confidence_score', 'mean')
).reset_index()

fraud_metrics.head(5)

,claim_id,fraud_detected,detection_count,avg_confidence
0,102,False,0,0.0
1,109,False,0,0.0
2,110,False,0,0.0
3,113,False,0,0.0
4,114,False,0,0.0


In [28]:
# Create policy-customer mapping
policy_customer_map = policies_df[['policy_id', 'customer_id']].drop_duplicates()

# Merge claims metrics with policy-customer mapping
policy_claims = pd.merge(
    policy_customer_map,
    claims_metrics,
    on='policy_id',
    how='left'
)

policy_claims.head()

,policy_id,customer_id,total_claims,total_claimed_amount,avg_claim_amount,claim_statuses
0,201,29,2.0,4730.78,2365.39,{'APPROVED': 2}
1,202,44,1.0,3369.93,3369.93,{'PENDING': 1}
2,203,35,1.0,7061.45,7061.45,{'PENDING': 1}
3,204,28,NaN,NaN,NaN,NaN
4,205,23,1.0,8567.58,8567.58,{'APPROVED': 1}


In [29]:
# Aggregate claims metrics at customer level
customer_claims = policy_claims.groupby('customer_id').agg({
    'total_claims': 'sum',
    'total_claimed_amount': 'sum',
    'avg_claim_amount': 'mean'
}).reset_index()

customer_claims.head()

,customer_id,total_claims,total_claimed_amount,avg_claim_amount
0,1,1.0,29430.41,29430.410
1,3,1.0,23396.02,23396.020
2,5,3.0,43622.79,12142.365
3,6,3.0,44645.06,19489.230
4,7,0.0,0.00,NaN


In [30]:
# Merge fraud data with claims and policy info
claims_fraud = pd.merge(
    claims_df[['claim_id', 'policy_id']],
    fraud_metrics,
    on='claim_id',
    how='left'
)

# Merge with policy-customer mapping
policy_fraud = pd.merge(
    claims_fraud,
    policy_customer_map,
    on='policy_id',
    how='left'
)

# Aggregate fraud metrics at customer level
customer_fraud_metrics = policy_fraud.groupby('customer_id').agg({
    'fraud_detected': 'max',
    'detection_count': 'sum',
    'avg_confidence': 'mean'
}).reset_index()

customer_fraud_metrics.head()

,customer_id,fraud_detected,detection_count,avg_confidence
0,1,NaN,0.0,NaN
1,3,NaN,0.0,NaN
2,5,False,0.0,0.0
3,6,False,0.0,0.0
4,8,NaN,0.0,NaN


In [33]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           50 non-null     int64         
 1   name                  50 non-null     object        
 2   age                   50 non-null     int64         
 3   email                 50 non-null     object        
 4   city                  50 non-null     object        
 5   registration_date     50 non-null     datetime64[ns]
 6   customer_tenure_days  50 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 2.9+ KB


In [34]:
# Merge policy metrics with claims metrics
customer_metrics = pd.merge(
    policy_metrics,
    customer_claims,
    on='customer_id',
    how='left'
)

# Merge with fraud metrics
customer_metrics_final = pd.merge(
    customer_metrics,
    customer_fraud_metrics,
    on='customer_id',
    how='left'
)

# Merge with customer data
customer_analytics = pd.merge(
    customers_df,
    customer_metrics_final,
    on='customer_id',
    how='left'
)

# Fill NaN values
numeric_cols = customer_analytics.select_dtypes(include=[np.number]).columns
customer_analytics[numeric_cols] = customer_analytics[numeric_cols].fillna(0)

# Calculate derived metrics
customer_analytics['claims_per_policy'] = customer_analytics['total_claims'] / customer_analytics['total_policies'].replace(0, np.nan)
customer_analytics['fraud_rate'] = customer_analytics['detection_count'] / customer_analytics['total_claims'].replace(0, np.nan)

customer_analytics.head()
customer_analytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           50 non-null     int64         
 1   name                  50 non-null     object        
 2   age                   50 non-null     int64         
 3   email                 50 non-null     object        
 4   city                  50 non-null     object        
 5   registration_date     50 non-null     datetime64[ns]
 6   customer_tenure_days  50 non-null     int64         
 7   total_policies        50 non-null     float64       
 8   active_policies       50 non-null     float64       
 9   avg_annual_premium    50 non-null     float64       
 10  total_coverage        50 non-null     float64       
 11  policy_types          41 non-null     object        
 12  policy_ids            41 non-null     object        
 13  total_claims          

In [32]:
# Export the customer analytics to a CSV file
output_path = '../outputs/customer_analytics_merged.csv'
customer_analytics.to_csv(output_path, index=False)
print(f"Customer analytics data has been exported to {output_path}")

# Optional: Display the first few rows to verify
print("\nFirst few rows of the exported data:")
customer_analytics.head()

Customer analytics data has been exported to ../outputs/customer_analytics_merged.csv

First few rows of the exported data:


,customer_id,name,age,email,city,registration_date,customer_tenure_days,total_policies,active_policies,avg_annual_premium,...,policy_types,policy_ids,total_claims,total_claimed_amount,avg_claim_amount,fraud_detected,detection_count,avg_confidence,claims_per_policy,fraud_rate
0,1,Kenneth Davis,56,customer1@email.com,New York,2023-12-30,588,1.0,1.0,4598.760,...,{'HEALTH': 1},[256],1.0,29430.41,29430.410,NaN,0.0,0.0,1.0,0.0
1,2,Deborah Moore,46,customer2@email.com,Los Angeles,2023-07-18,753,0.0,0.0,0.000,...,NaN,NaN,0.0,0.00,0.000,NaN,0.0,0.0,NaN,NaN
2,3,Joseph Rodriguez,60,customer3@email.com,Philadelphia,2022-12-31,952,1.0,1.0,5543.190,...,{'BUSINESS': 1},[257],1.0,23396.02,23396.020,NaN,0.0,0.0,1.0,0.0
3,4,Michael Baker,38,customer4@email.com,Philadelphia,2020-05-01,1926,0.0,0.0,0.000,...,NaN,NaN,0.0,0.00,0.000,NaN,0.0,0.0,NaN,NaN
4,5,Paul Garcia,36,customer5@email.com,Phoenix,2023-05-23,809,2.0,0.0,4145.435,...,"{'LIFE': 1, 'BUSINESS': 1}","[259, 261]",3.0,43622.79,12142.365,False,0.0,0.0,1.5,0.0


## Now, use this new dataset for EDA and performing the required analysis